<a href="https://colab.research.google.com/github/zczllw5/Spaceship_Titanic/blob/main/Spaceship_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

**Overview**
1. Understand the shape of the data(Histogram, box plots, ect.)
2. Data cleaning
3. Data exploration
4. Feature Engineering
5. Preprocessing for Model
6. Basic model building
7. Model tuning
8. Ensumble model building
9. Results

# Understand the shape of the data

In [ ]:
training = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

training['train_test'] = 1
test['train_test'] = 0
test['Transported'] = np.NaN

all_data = pd.concat([training,test])

<ipython-input-243-6cc8be82b79e>:8: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  all_data = pd.concat([training,test])


2 & 3 Data Exploration

In [ ]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12970 entries, 0 to 4276
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   12970 non-null  object 
 1   HomePlanet    12682 non-null  object 
 2   CryoSleep     12660 non-null  object 
 3   Cabin         12671 non-null  object 
 4   Destination   12696 non-null  object 
 5   Age           12700 non-null  float64
 6   VIP           12674 non-null  object 
 7   RoomService   12707 non-null  float64
 8   FoodCourt     12681 non-null  float64
 9   ShoppingMall  12664 non-null  float64
 10  Spa           12686 non-null  float64
 11  VRDeck        12702 non-null  float64
 12  Name          12676 non-null  object 
 13  Transported   8693 non-null   float64
 14  train_test    12970 non-null  int64  
dtypes: float64(7), int64(1), object(7)
memory usage: 1.6+ MB


In [ ]:
# quick look at data types and null counts
training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
 14  train_test    8693 non-null   int64  
dtypes: bool(1), float64(6), int64(1), object(7)
memory usage: 959.4+ KB


In [ ]:
#To better understaning, we using .describe() method. It gives us a understanding of central tendencies of the data
training.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,train_test
count,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000,8693.0
mean,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791,1.0
std,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
75%,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000,1.0
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000,1.0


In [ ]:
#qucik way to separet the coloumns
training.describe().columns

Index(['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'train_test'],
      dtype='object')

In [ ]:
#Seperate nummeric columns and categorical columns
df_num = training[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']]
df_cat = training[['Cabin',"HomePlanet","CryoSleep","Destination", "VIP", "Transported"]]

In [ ]:
#distribution for all nummeric variables
for i in df_num:
  plt.hist(df_num[i])
  plt.title(i)
  plt.show()


In [ ]:
print(df_num.corr())
sns.heatmap(df_num.corr())

In [ ]:
# compare transported rate across Age, SibSp, Parch, and Fare
pd.pivot_table(training, index = 'Transported', values = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'])

,Age,FoodCourt,RoomService,ShoppingMall,Spa,VRDeck
Transported,,,,,,
False,29.922858,382.615930,389.266066,167.566217,564.382666,543.629822
True,27.748834,532.691984,63.098021,179.829972,61.675531,69.148131


In [ ]:
# Comparing survival and each of these categorical variables
# "HomePlanet","CryoSleep","Destination", "VIP"
print(pd.pivot_table(training, index = 'Transported', columns = "HomePlanet", aggfunc ='count'))
print()
print(pd.pivot_table(training, index = 'Transported', columns = "CryoSleep", aggfunc ='count'))
print()
print(pd.pivot_table(training, index = 'Transported', columns = "Destination", aggfunc ='count'))
print()
print(pd.pivot_table(training, index = 'Transported', columns = 'VIP', aggfunc ='count'))

Data Engineering


1.   Drop 'Name', 'PassengerId', 'Cabin'
2.   One hot key encoding categorical features:'HomePlanet', 'CryoSleep', 'Destination', 'VIP'
3.   Seperate Transported as Y_val
4.   Deal with missing data: drow categorical missing data and fill numerical data with a mean value


In [ ]:
# 1. Drop 'Name', 'PassengerId', 'Cabin'

all_data = all_data.drop(columns=[ 'Name', 'PassengerId', 'Cabin'])
all_data

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,train_test
0,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,0.0,1
1,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,1.0,1
2,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,0.0,1
3,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,0.0,1
4,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
4272,Earth,True,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,NaN,0
4273,Earth,False,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,NaN,0
4274,Mars,True,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,NaN,0
4275,Europa,False,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0,NaN,0


In [ ]:
all_data.columns

Index(['HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP', 'RoomService',
       'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Transported',
       'train_test'],
      dtype='object')

In [ ]:

# Perform one-hot encoding on a categorical column
all_data = pd.get_dummies(all_data, columns=['HomePlanet',"CryoSleep","VIP","Destination"])

# Print the encoded data
print(all_data)

       Age  RoomService  FoodCourt  ShoppingMall     Spa  VRDeck  Transported  \
0     39.0          0.0        0.0           0.0     0.0     0.0          0.0   
1     24.0        109.0        9.0          25.0   549.0    44.0          1.0   
2     58.0         43.0     3576.0           0.0  6715.0    49.0          0.0   
3     33.0          0.0     1283.0         371.0  3329.0   193.0          0.0   
4     16.0        303.0       70.0         151.0   565.0     2.0          1.0   
...    ...          ...        ...           ...     ...     ...          ...   
4272  34.0          0.0        0.0           0.0     0.0     0.0          NaN   
4273  42.0          0.0      847.0          17.0    10.0   144.0          NaN   
4274   NaN          0.0        0.0           0.0     0.0     0.0          NaN   
4275   NaN          0.0     2680.0           0.0     0.0   523.0          NaN   
4276  43.0          0.0        0.0           0.0     0.0     0.0          NaN   

      train_test  HomePlane

In [ ]:
missing_data = all_data.isnull().sum()
print(missing_data)

Age                           270
RoomService                   263
FoodCourt                     289
ShoppingMall                  306
Spa                           284
VRDeck                        268
Transported                  4277
train_test                      0
HomePlanet_Earth                0
HomePlanet_Europa               0
HomePlanet_Mars                 0
CryoSleep_False                 0
CryoSleep_True                  0
VIP_False                       0
VIP_True                        0
Destination_55 Cancri e         0
Destination_PSO J318.5-22       0
Destination_TRAPPIST-1e         0
dtype: int64


Missing data


*   Age: fillna with mean
*   Roomservice, FoodCourt, ShoppingMall, Spa, VRDeck not normadistribution: fill with 0





In [ ]:

all_data.loc[:,'Age'] = all_data['Age'].fillna(np.mean(all_data['Age']))

In [ ]:
columns_to_fill = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
all_data.loc[:, columns_to_fill] = all_data.loc[:, columns_to_fill].fillna(0)

In [ ]:
missing_data = all_data.isnull().sum()
print(missing_data)

Age                             0
RoomService                     0
FoodCourt                       0
ShoppingMall                    0
Spa                             0
VRDeck                          0
Transported                  4277
train_test                      0
HomePlanet_Earth                0
HomePlanet_Europa               0
HomePlanet_Mars                 0
CryoSleep_False                 0
CryoSleep_True                  0
VIP_False                       0
VIP_True                        0
Destination_55 Cancri e         0
Destination_PSO J318.5-22       0
Destination_TRAPPIST-1e         0
dtype: int64


In [ ]:
#add a new coloumn: if they are grouped or not?


In [ ]:
all_data['train_test'] == 1


0        True
1        True
2        True
3        True
4        True
        ...  
4272    False
4273    False
4274    False
4275    False
4276    False
Name: train_test, Length: 12970, dtype: bool

#Scaling

In [ ]:
#only scale numerical features
from sklearn.preprocessing import StandardScaler

scale = StandardScaler()

all_data_scaled = all_data.copy()

all_data_scaled[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']] = scale.fit_transform(all_data_scaled[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']])
all_data_scaled

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,train_test,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_False,CryoSleep_True,VIP_False,VIP_True,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,7.184543e-01,-0.340290,-0.281822,-0.292365,-0.269707,-0.257100,0.0,1,0,1,0,1,0,1,0,0,0,1
1,-3.352005e-01,-0.170439,-0.276082,-0.249566,0.221040,-0.219449,1.0,1,1,0,0,1,0,1,0,0,0,1
2,2.053084e+00,-0.273285,1.998823,-0.292365,5.732776,-0.215170,0.0,1,0,1,0,1,0,0,1,0,0,1
3,2.969924e-01,-0.340290,0.536429,0.342766,2.706059,-0.091947,0.0,1,0,1,0,1,0,1,0,0,0,1
4,-8.971498e-01,0.131863,-0.237179,-0.033861,0.235342,-0.255389,1.0,1,1,0,0,1,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,3.672360e-01,-0.340290,-0.281822,-0.292365,-0.269707,-0.257100,NaN,0,1,0,0,0,1,1,0,0,0,1
4273,9.291853e-01,-0.340290,0.258364,-0.263262,-0.260768,-0.133877,NaN,0,1,0,0,1,0,1,0,0,0,1
4274,-2.495556e-16,-0.340290,-0.281822,-0.292365,-0.269707,-0.257100,NaN,0,0,0,1,0,1,1,0,1,0,0
4275,-2.495556e-16,-0.340290,1.427386,-0.292365,-0.269707,0.190440,NaN,0,0,1,0,1,0,1,0,0,0,0


In [ ]:
#Split train and test again
X_train = all_data_scaled[all_data_scaled.train_test == 1].drop('train_test', axis = 1)
X_train = X_train.drop('Transported', axis = 1)
print(X_train)

In [ ]:
X_test = all_data_scaled[all_data_scaled.train_test == 0].drop('train_test', axis = 1)
X_test = X_test.drop('Transported', axis = 1)
print(X_test)

In [ ]:
y_train = all_data_scaled[all_data_scaled.train_test == 1].Transported
print(y_train)

#Model Building


*   Support Vector Classifier



In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
svc = SVC(max_iter=10000)  # Increase the value as needed
svc.fit(X_train, y_train)

cv = cross_val_score(svc,X_train,y_train,cv=5)
print(cv)
print(cv.mean())

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[0.78953422 0.77975848 0.7786084  0.79976985 0.79976985]
0.7894881586774936


#Make predictions



In [ ]:
 # Predict using the trained model
y_pred = svc.predict(X_test)

In [ ]:
y_pred.shape

(4277,)

# Submission


In [ ]:
# Convert NumPy array to a pandas DataFrame
y_pred = pd.DataFrame(y_pred, columns=['Transported'])
y_pred

,Transported
0,1.0
1,0.0
2,1.0
3,1.0
4,0.0
...,...
4272,1.0
4273,0.0
4274,1.0
4275,1.0


In [ ]:
# Convert values to True and False
y_pred['Transported'] = y_pred['Transported'].replace({1.0: True, 0.0: False})
y_pred

,Transported
0,True
1,False
2,True
3,True
4,False
...,...
4272,True
4273,False
4274,True
4275,True


In [ ]:
y_pred_concated = pd.concat([test.PassengerId,y_pred],axis=1)
y_pred_concated

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


In [ ]:
# generate csv file
y_pred_concated.to_csv('submission_svc.csv', index=False)